In [35]:
import os
import gradio as gr
import google.generativeai as genai
from dotenv import load_dotenv
import pandas as pd
import webbrowser

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key
api_key = os.getenv("Gemini_API_Key")
if not api_key:
    raise ValueError("API key not found. Ensure 'Gemini_API_Key' is correctly set in the .env file.")

genai.configure(api_key=api_key)

# Load the wine dataset
nlp_file_path = "../Resources/nlp_3.1_final.csv"
wine_df = pd.read_csv(nlp_file_path)

# Extract unique attribute options
color_options = wine_df["color"].explode().unique().tolist()
aroma_options = wine_df["aromas"].explode().unique().tolist()
flavor_options = [
    "Citrus", "Berries", "Stone Fruits", "Tropical Fruits", "Dried Fruits",
    "Floral", "Herbal", "Vegetal", "Spices", "Woody", "Nutty",
    "Toasty", "Earthy", "Mineral", "Chocolate & Coffee", "Caramel & Vanilla",
    "Sweet", "Savory", "Oaky", "Smoky"
]
body_options = ["Light-bodied", "Medium-bodied", "Full-bodied"]

# Adjustable weights for aroma
aroma_weight = 0.5

# Finalized Wine Color Options with Descriptions
wine_colors = {
    "Red": "Ranges from light (garnet) to deep (opaque) reds. Full-bodied reds tend to have higher tannins.",
    "White": "Ranges from clear to golden hues, with fuller-bodied whites having a richer color due to oak aging.",
    "Rosé": "Varies from pale salmon to magenta, depending on grape variety and skin contact time.",
    "Orange": "A white wine made with extended skin contact, giving it a deep amber color.",
    "Sparkling": "Can be white, rosé, or even red with effervescence from natural carbonation."
}

# Aroma Options (Category - Subcategory format)
aroma_options = [
    "Fruity White Wine - Citrus",
    "Fruity White Wine - White Berries",
    "Fruity White Wine - Pome Fruits",
    "Fruity White Wine - Stone Fruits",
    "Fruity White Wine - Tropical Fruits",
    "Fruity White Wine - Botrytized",
    "Fruity Red Wine - Tropical Fruits",
    "Fruity Red Wine - Red Berries",
    "Fruity Red Wine - Stone Fruits",
    "Fruity Red Wine - Fortified",
    "Floral - White Flowers",
    "Floral - Colored Flowers",
    "Vegetal - Vegetables",
    "Vegetal - Fresh Herbs",
    "Vegetal - Dried Herbs",
    "Vegetal - Leaves",
    "Maturation in Oak Barrel - Woods",
    "Maturation in Oak Barrel - Nuts",
    "Maturation in Oak Barrel - Spices",
    "Maturation in Oak Barrel - Toasted"
]

# Price Selection (Dropdown with explicit ranges)
price_options = {
    "$1 - $9.99": "Budget-friendly wines, great for casual drinking.",
    "$10 - $99.99": "Standard quality wines, suitable for most occasions.",
    "$100 - $999.99": "Premium and high-quality wines for special occasions.",
    "$1000+": "Rare, collectible, and investment-grade wines."
}

# Wine Recommendation Function
def get_wine_recommendation(wine_color, flavor_profile, aroma, sweetness, body, price_range):
    """First searches the dataset for a wine recommendation. If no match, use Gemini AI."""
    
    # Filter dataset based on user inputs
    filtered_wines = wine_df[
        wine_df["color"].apply(lambda x: wine_color in x) &
        wine_df["flavors"].apply(lambda x: flavor_profile in x) &
        wine_df["aromas"].apply(lambda x: aroma in x)
    ]
    
    # If we find a matching wine in the dataset
    if not filtered_wines.empty:
        recommended_wine = filtered_wines.sample(1).iloc[0]
        wine_name = recommended_wine['Name']
        wine_recommendation = f"""
        **Recommended Wine:**  
        **{wine_name}**  

        **Attributes:**  
        - **Color:** {recommended_wine['color']}  
        - **Flavor Profile:** {flavor_profile}  
        - **Aroma:** {recommended_wine['aromas']}  
        - **Sweetness Level:** {sweetness}  
        - **Body:** {body}  
        - **Price:** {price_range}  
        """
    else:
        # If no exact match, use Gemini AI
        prompt = f"""
        You are a wine expert. Based on the following attributes, recommend a wine:
        - Wine Color: {wine_color}
        - Flavor Profile: {flavor_profile}
        - Aroma: {aroma}
        - Sweetness Level: {sweetness}
        - Body: {body}
        - Price Range: {price_range}
        Provide the best wine recommendation.
        """
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(prompt)
    wine_recommendation = response.text
    wine_name = response.text.split('\n')[2].strip('**') if '\n' in response.text and len(response.text.split('\n')) > 2 else wine_color  # Extract the first line as the wine name

    return wine_recommendation, wine_name

# Function to open the Google Search link when button is clicked
def open_google_search(wine_name):
    search_url = f"https://www.google.com/search?q={wine_name.replace(' ', '+')}"
    webbrowser.open(search_url)

# Build Gradio UI
with gr.Blocks() as demo:
    wine_name_state = gr.State(value="")
    
    inputs = [
        gr.Dropdown(list(wine_colors.keys()), label="Wine Color"),
        gr.Dropdown(flavor_options, label="Flavor Profile"),
        gr.Dropdown(aroma_options, label="Aroma Preference"),
        gr.Slider(0, 10, step=1, label="Sweetness Level (0 = Dry, 10 = Very Sweet)"),
        gr.Dropdown(body_options, label="Wine Body"),
        gr.Dropdown(list(price_options.keys()), label="Price Range"),
    ]
    
    recommendation_output = gr.Textbox(label="Recommendations")
    submit_btn = gr.Button("Submit", variant="primary", interactive=True, elem_id="submit-btn", scale=1)
    search_button = gr.Button("Find My Wine", variant="secondary", interactive=True, elem_id="search-btn", scale=1)
    
    
    submit_btn.click(get_wine_recommendation, inputs=inputs, outputs=[recommendation_output, wine_name_state])
    
    search_button.click(open_google_search, inputs=[wine_name_state], outputs=[])

demo.launch()


* Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.
